# VGG16

# Google drive mount and extract dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Dataset/data.zip" -d "/content/data"

In [ ]:
!unzip "/content/drive/MyDrive/Dataset/data.zip"

In [6]:
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

# Loading Data generator function

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
# Define data generators for training, validation, and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Zoom transformations
    horizontal_flip=True  # Horizontal flips
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for testing data

# Define paths to your train, test, and validation directories
train_dir = 'train/'
validation_dir = 'validation/'
test_dir = 'test/'

# Set up data generators to read images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,  # Number of images in each batch
    class_mode='categorical'  # Since you have multiple classes
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5180 images belonging to 38 classes.
Found 1742 images belonging to 38 classes.
Found 1748 images belonging to 38 classes.


# Model

In [9]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [10]:
# Load pre-trained Xception model (excluding top layers)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

NUM_CLASSES = 38

# Add custom top layers for fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(1024, activation='relu')(x)  # Fully connected layer with 1024 units
predictions = Dense(NUM_CLASSES, activation='softmax')(x)  # Output layer with softmax activation

# Combine base model with custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
# model.summary()

83683744/83683744 [==============================] - 3s 0us/step


# Check GPU availability

In [11]:
import tensorflow as tf

# Check if GPU is available
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Check TensorFlow GPU support
print("TensorFlow GPU Support:", tf.test.is_built_with_cuda())

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow GPU Support: True


# Train

In [12]:
# Define the number of training and validation steps per epoch
train_steps_per_epoch = len(train_generator)
validation_steps_per_epoch = len(validation_generator)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=100,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch
)

# Evaluate the model on the test set
test_steps_per_epoch = len(test_generator)
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps_per_epoch)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/100
 32/162 [====>.........................] - ETA: 1:00 - loss: 3.5685 - accuracy: 0.0853

KeyboardInterrupt: 